# Opacus Solution for the Heart Disease Prediction Hackathon

In [ ]:
!pip install antigranular

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.7 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.8.2
    Uninstalling pydantic-2.8.2:
      Successfully uninstalled pydantic-2.8.2
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
google-colab 1.

In [ ]:
import antigranular as ag
session = ag.login(<client_id>,<client_secret>, competition = "Heart Disease Prediction Hackathon")

Dataset "Heart Disease Prediction Hackathon Dataset" loaded to the kernel as heart_disease_prediction_hackathon_dataset
Key Name                       Value Type     
---------------------------------------------
train_y                        PrivateDataFrame
train_x                        PrivateDataFrame
test_x                         DataFrame      

Connected to Antigranular server session id: 824e44ef-609d-4509-8f5e-50ecd1a22ce6, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


In [ ]:
# check initial privacy budget
session.privacy_odometer()

{'total_epsilon_used': 1.0968392252096653,
 'total_delta_used': 0.10000010000000001,
 'library_costs': {'op_pandas': {'total_delta': 0.0,
   'total_epsilon': 1.1,
   'total_requests': 48},
  'op_opacus': {'total_delta': 0.10000010000000001,
   'total_epsilon': -0.003160774790334747,
   'total_requests': 2}},
 'dataset_costs': {'Heart Disease Prediction Hackathon Dataset': {'delta': 0.10000010000000001,
   'eps': 1.0968392252096653}}}

In [ ]:
# see data set info
%%ag
x_train = heart_disease_prediction_hackathon_dataset['train_x']
ag_print(x_train.info())
y_train = heart_disease_prediction_hackathon_dataset['train_y']
ag_print(y_train.info())

+----+----------+-------------+---------------+---------+------------+
|    | Column   | numerical   | categorical   | dtype   | bounds     |
|----+----------+-------------+---------------+---------+------------|
|  0 | age      | True        | False         | int64   | (21, 86)   |
|  1 | sex      | True        | False         | int64   | (0, 1)     |
|  2 | bp       | True        | False         | int64   | (80, 215)  |
|  3 | ch       | True        | False         | int64   | (102, 597) |
|  4 | bs       | True        | False         | int64   | (67, 157)  |
|  5 | phr      | True        | False         | int64   | (62, 222)  |
+----+----------+-------------+---------------+---------+------------+

None
+----+-----------+-------------+---------------+---------+----------+
|    | Column    | numerical   | categorical   | dtype   | bounds   |
|----+-----------+-------------+---------------+---------+----------|
|  0 | condition | True        | False         | int64   | (0, 1)   |
+---

In [ ]:
%%ag
# look at the test dataset
x_test = heart_disease_prediction_hackathon_dataset['test_x']
ag_print(x_test)
number_of_nans = x_test.isna().sum().sum()
ag_print("Number of Nans in the dataset = ", number_of_nans)

      age  sex   bp   ch   bs  phr
0      71    1  128  326   95  117
1      61    1  153  270   98  123
2      59    1  113  236  106  181
3      69    0  109  151  109  108
4      55    0  137  235  101  150
...   ...  ...  ...  ...  ...  ...
1995   60    1  128  261  112  143
1996   50    1  143  216   94  100
1997   64    1  120  172   87  142
1998   56    1  158  294   82  144
1999   69    0  117  559  112  157

[2000 rows x 6 columns]
Number of Nans in the dataset =  0



# Preprocessing

I used a high privacy budget (EPSILON = 1) as a not accurate scaling may influence badly the accuracy of the model

In [ ]:
%%ag
from op_pandas import standard_scaler

EPSILON = 1.
x_train_scaled = standard_scaler(x_train, eps=EPSILON)
x_train_scaled.info()

+----+----------+-------------+---------------+---------+------------------------------------------+
|    | Column   | numerical   | categorical   | dtype   | bounds                                   |
|----+----------+-------------+---------------+---------+------------------------------------------|
|  0 | age      | True        | False         | float64 | (-3.599371111901545, 3.354123011367128)  |
|  1 | sex      | True        | False         | float64 | (-1.4498749957255392,                    |
|    |          |             |               |         | 0.6954137186736699)                      |
|  2 | bp       | True        | False         | float64 | (-3.2881900113347124, 5.286276908283066) |
|  3 | ch       | True        | False         | float64 | (-2.890976830822871, 6.863751715429815)  |
|  4 | bs       | True        | False         | float64 | (-2.2776804909956923,                    |
|    |          |             |               |         | 3.4491071819273342)              

In [ ]:
%%ag
from op_pandas import PrivateDataFrame
# apply the scaling to the test dataset
EPSILON = 1.
x_test_scaled = standard_scaler(PrivateDataFrame(x_test), eps=EPSILON)

# Model using Pytorch and Opacus

In [ ]:
%%ag
import torch.nn as nn
from torch import optim

# creating a Neural Network for Binary Classification
model = nn.Sequential(
    nn.Linear(6, 32),
    nn.ReLU(),
    nn.Linear(32, 64),
    nn.ReLU(),
    nn.Linear(64, 128),
    nn.ReLU(),
    nn.Linear(128, 1),
    nn.Sigmoid(),
)

# Stochastic Gradient Descent Optimizer
optimizer = optim.SGD(model.parameters(), lr=0.005)

# loss function is Binary Cross Entropy
loss_fn = nn.BCELoss()

# Hyper-parameters
EPOCHS = 50
BATCH_SIZE = 64

In [ ]:
%%ag
from op_opacus import PrivateDPDataLoader

# Creation of the Private Data Loader
data_loader = PrivateDPDataLoader.from_private_dataframe(
    [x_train_scaled, y_train], dtypes=[torch.float, torch.float], batch_size=BATCH_SIZE
)

In [ ]:
%%ag
from op_opacus import PrivacyEngine

# Instantiation of the Privacy Engine from Opacus
privacy_engine = PrivacyEngine()

# Privacy Parameters for the training
EPSILON = 0.9
DELTA = 1E-6

# Specification for the DP-SGD
privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=data_loader,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    epochs=EPOCHS,
    max_grad_norm=1.,
)

/usr/local/lib/python3.10/site-packages/opacus_internal/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/opacus_internal/accountants/analysis/prv/prvs.py:50: RuntimeWarning: invalid value encountered in log
  z = np.log((np.exp(t) + q - 1) / q)



# Training

In [ ]:
%%ag
from op_opacus import TrainModel

# function for the training
def train_callable(model, optimizer, data, loss_function):
    inputs = data[0]
    labels = data[1]
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

train_model = TrainModel(privacy_engine, loss_fn)
train_model.train(train_callable, verbose=2)

/code/dependencies/op_opacus/op_opacus/train_model.py:34: UserWarning: Loss object is not made private, due to which losses cant be shown for each epoch
  warnings.warn("Loss object is not made private, due to which losses cant be shown for each epoch",



Epoch 1 completed.
 Epsilon used: 0.12536612979298495,For target delta: 1e-06
Average loss for this epoch : None, time taken: 10.10487769098836 seconds.

Epoch 2 completed.
 Epsilon used: 0.17432505961204214,For target delta: 1e-06
Average loss for this epoch : None, time taken: 9.588097275991458 seconds.

Epoch 3 completed.
 Epsilon used: 0.2125355561850568,For target delta: 1e-06
Average loss for this epoch : None, time taken: 9.497777078009676 seconds.

Epoch 4 completed.
 Epsilon used: 0.2451059785638144,For target delta: 1e-06
Average loss for this epoch : None, time taken: 9.389127100992482 seconds.

Epoch 5 completed.
 Epsilon used: 0.2740402113400341,For target delta: 1e-06
Average loss for this epoch : None, time taken: 9.694557037000777 seconds.

Epoch 6 completed.
 Epsilon used: 0.30037449002412536,For target delta: 1e-06
Average loss for this epoch : None, time taken: 9.493035966006573 seconds.

Epoch 7 completed.
 Epsilon used: 0.3247284520373031,For target delta: 1e-06
Av

/usr/local/lib/python3.10/site-packages/opacus_internal/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/opacus_internal/accountants/analysis/prv/prvs.py:50: RuntimeWarning: invalid value encountered in log
  z = np.log((np.exp(t) + q - 1) / q)



# Send the result

In [ ]:
%%ag
from op_opacus import ApplyModel

# test model is the dp model after training
test_model = ApplyModel(privacy_engine=privacy_engine)
y_pred = test_model.apply_model_private(x_test_scaled,
                                        dtype=torch.float,
                                        output_col_names=["output"],
                                        output_bounds = {"output": (0,1)}) # output of Sigmoid is in (0,1)

# Note that the predictions are a float scalar so we need to transform it in integers
def f(x: float) -> float:
  if x > 0.5:
    return 1
  else:
    return 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))

In [ ]:
%%ag
result = submit_predictions(y_pred)

score: {'leaderboard': 0.704567982034695, 'logs': {'BIN_ACC': 0.7190432121054476, 'LIN_EPS': -0.014475230070752644}}

